# Деревья решений и метод ближайших соседей в задачах классификации. Метрики оценки качества обучения.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder

#Деревья решений
from sklearn.tree import DecisionTreeClassifier
#Разбивка набора данных на обучающую и тестовую выборки
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score

from sklearn.metrics import classification_report

from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import recall_score

# используем .dot формат для визуализации дерева
from sklearn.tree import export_graphviz

from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import cross_val_score

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings("ignore")

### Рассмотрим еще один набор данных.

In [2]:
path1 = 'cardio.csv'

In [3]:
df = pd.read_csv(path1, index_col='id', sep=';')

In [4]:
df.shape

(70000, 12)

In [5]:
df.head()

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
id,,,,,,,,,,,,
0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,17474,1,156,56.0,100,60,1,1,0,0,0,0


В данных присутствует 3 типа признаков:

- *Objective*: фактическая информация;
- *Examination*: результаты медицинского обследования;
- *Subjective*: информация представленная пациентом.

| Feature | Variable Type | Variable      | Value Type |
|---------|--------------|---------------|------------|
| Age | Objective Feature | age | int (days) |
| Height | Objective Feature | height | int (cm) |
| Weight | Objective Feature | weight | float (kg) |
| Gender | Objective Feature | gender | categorical code |
| Systolic blood pressure | Examination Feature | ap_hi | int |
| Diastolic blood pressure | Examination Feature | ap_lo | int |
| Cholesterol | Examination Feature | cholesterol | 1: normal, 2: above normal, 3: well above normal |
| Glucose | Examination Feature | gluc | 1: normal, 2: above normal, 3: well above normal |
| Smoking | Subjective Feature | smoke | binary |
| Alcohol intake | Subjective Feature | alco | binary |
| Physical activity | Subjective Feature | active | binary |
| Presence or absence of cardiovascular disease | Target Variable | cardio | binary |

Возраст представлен в днях от рождения.

Целевой признак - cardio, которые означает, что человек в группе риска сердечных заболеваний.

Выделим целевой признак и поделим набор данных в соотношении 70:30.

In [6]:
target = df['cardio']
df.drop('cardio', axis = 1, inplace = True)

In [7]:
X_train, X_holdout, y_train, y_holdout = train_test_split(df, target, test_size=0.3,
                                                          random_state=17)

In [8]:
X_train.shape, X_holdout.shape, y_train.shape, y_holdout.shape

((49000, 11), (21000, 11), (49000,), (21000,))

## Задание в стиле "free ride"

Используя полученные наборы данных для train и test необходимо:
- получить максимальные значения метрик accuracy и recall на тестовой выборке с использованием дерева решений;
- получить максимальные значения метрик accuracy и recall на тестовой выборке с использованием метода ближайших соседей;

Попробуйте добавить новые признаки в модель для улучшения показателя метрик, например:
- возраст в годах - поделить признак возраст на 365.25 ;create "age in years" (full age) - the remainder of dividing age in days by 365.25
- создать 3 бинарных признака, основанных на признаке cholesterol используя get_dummies или oneHotEncoder (изучить самостоятельно);
-  создать 3 бинарных признака основанных на признаке gluc;
- и т.д.

# 1 часть

## Дерево решений

### Entropy

In [9]:
tree = DecisionTreeClassifier(criterion='entropy', max_depth=5, random_state=17)
tree.fit(X_train, y_train) #обучение
tree_pred = tree.predict(X_holdout) #предсказание

In [10]:
report = classification_report(y_holdout, tree.predict(X_holdout))
print (report)

              precision    recall  f1-score   support

           0       0.70      0.80      0.75     10490
           1       0.77      0.66      0.71     10510

    accuracy                           0.73     21000
   macro avg       0.74      0.73      0.73     21000
weighted avg       0.74      0.73      0.73     21000



### Accuracy

In [11]:
accuracy_score(y_holdout, tree_pred) # с помощью дерева решений

0.7308571428571429

### Entropy + увеличение глубины дерева

In [12]:
tree = DecisionTreeClassifier(criterion='entropy', max_depth=15, random_state=17)
tree.fit(X_train, y_train)
tree_pred = tree.predict(X_holdout)

In [13]:
report = classification_report(y_holdout, tree.predict(X_holdout))
print (report)

              precision    recall  f1-score   support

           0       0.70      0.75      0.72     10490
           1       0.73      0.68      0.70     10510

    accuracy                           0.71     21000
   macro avg       0.71      0.71      0.71     21000
weighted avg       0.71      0.71      0.71     21000



### Gini + увеличение глубины дерева

In [14]:
tree = DecisionTreeClassifier(criterion='gini', max_depth=15, random_state=17)
tree.fit(X_train, y_train)
tree_pred = tree.predict(X_holdout)

In [15]:
report = classification_report(y_holdout, tree.predict(X_holdout))
print (report)

              precision    recall  f1-score   support

           0       0.69      0.73      0.71     10490
           1       0.72      0.68      0.70     10510

    accuracy                           0.71     21000
   macro avg       0.71      0.71      0.71     21000
weighted avg       0.71      0.71      0.71     21000



### Кросс-валидация

In [16]:
tree_param = {'max_depth': range(1,10,2),'max_features': range(4,7)}
tree_gridcv = GridSearchCV(tree, tree_param, cv=5, n_jobs=-1, verbose=True)
tree_gridcv.fit(X_train, y_train)

Fitting 5 folds for each of 15 candidates, totalling 75 fits


GridSearchCV(cv=5,
             estimator=DecisionTreeClassifier(max_depth=15, random_state=17),
             n_jobs=-1,
             param_grid={'max_depth': range(1, 10, 2),
                         'max_features': range(4, 7)},
             verbose=True)

In [17]:
tree_gridcv.best_params_

{'max_depth': 7, 'max_features': 6}

In [18]:
tree_gridcv.best_score_

0.7281020408163266

In [19]:
recall_score(y_holdout, tree_gridcv.predict(X_holdout))

0.6798287345385348

In [20]:
accuracy_score(y_holdout, tree_gridcv.predict(X_holdout))

0.7254285714285714

Лучшие параметры модели: глубина дерева - 7, количество признаков - 6. <br>
Оценка на кросс-валидации ~ 0.73,<br>
оценка recall на отложенной выборке ~ 0.67,<br> 
оценка accuracy на отложенной выборке ~ 0.73.

### Визуализация дерева

In [21]:
tree = DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=17)
tree.fit(X_train, y_train) #обучение

DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=17)

In [22]:
export_graphviz(tree, #feature_names=['age', 'gender', 'height','weight', 'smoke', 'alco', 'active'], 
out_file='tree3.dot', filled=True)
# для этого понадобится библиотека pydot (pip install pydot)
!dot -Tpng 'tree3.dot' -o 'tree3.png'

'dot' is not recognized as an internal or external command,
operable program or batch file.


![img](https://sun9-70.userapi.com/impg/799-2JM_ve93ePABVTXRJvHZItq6jFTo0mOd2w/kbl9O7DoujU.jpg?size=957x329&quality=96&sign=3c94e82dcc8e89f67340e11c8c8f8d06&type=album)

## Метод ближайших соседей

In [23]:
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_train, y_train)
pred_knn = knn.predict(X_holdout)

In [24]:
report = classification_report(y_holdout, pred_knn)
print (report)

              precision    recall  f1-score   support

           0       0.67      0.78      0.72     10490
           1       0.73      0.61      0.67     10510

    accuracy                           0.70     21000
   macro avg       0.70      0.70      0.69     21000
weighted avg       0.70      0.70      0.69     21000



In [25]:
accuracy_score(y_holdout, pred_knn)

0.6953809523809524

### Кросс-валидация

In [26]:
knn_pipe = Pipeline([('scaler', StandardScaler()), ('knn', KNeighborsClassifier(n_jobs=-1))])

In [27]:
knn_params = {'knn__n_neighbors': range(17, 20)}
knn_grid = GridSearchCV(knn_pipe, knn_params, cv=5, n_jobs=-1, verbose=True)
knn_grid.fit(X_train, y_train)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('knn',
                                        KNeighborsClassifier(n_jobs=-1))]),
             n_jobs=-1, param_grid={'knn__n_neighbors': range(17, 20)},
             verbose=True)

In [28]:
knn_grid.best_params_

{'knn__n_neighbors': 17}

In [29]:
knn_grid.best_score_

0.6625918367346938

In [30]:
recall_score(y_holdout, knn_grid.predict(X_holdout))

0.631208372978116

In [31]:
accuracy_score(y_holdout, knn_grid.predict(X_holdout))

0.662952380952381

Лучшие параметры модели: соседей - 17. <br>
Оценка на кросс-валидации ~ 0.66,<br>
оценка recall на отложенной выборке ~ 0.63,<br> 
оценка accuracy на отложенной выборке ~ 0.66.

# 2 часть


## Признак "возраст в годах", где возраст поделить на 365.25

In [32]:
AgeYear = pd.DataFrame({'Возраст в годах': df['age']/365.25})

## 3 бинарных признака, основанных на признаке cholesterol, используя get_dummies или oneHotEncoder

### oneHotEncoder

In [ ]:
from sklearn.preprocessing import OneHotEncoder
Cholesterol = pd.DataFrame({'Холестерин': df['cholesterol']}) 
onehotencoder = OneHotEncoder(categories='cholesterol')
x = onehotencoder.fit_transform(Cholesterol).toarray()

In [ ]:
Cholesterol = pd.DataFrame(df, columns=['cholesterol'])
X = Cholesterol.values
cholesterol = np.unique(X[:,0])
ohe = OneHotEncoder(categories=[cholesterol])
X = ohe.fit_transform(X).toarray()
print (X)

### get_dummies

In [ ]:
s=pd.Series(df['cholesterol'])
Cholesterol=pd.get_dummies(s)
print(Cholesterol)

       1  2  3
id            
0      1  0  0
1      0  0  1
2      0  0  1
3      1  0  0
4      1  0  0
...   .. .. ..
99993  1  0  0
99995  0  1  0
99996  0  0  1
99998  1  0  0
99999  0  1  0

[70000 rows x 3 columns]


## 3 бинарных признака, основанных на признаке gluc

In [ ]:
s=pd.Series(df['gluc'])
Cholesterol=pd.get_dummies(s)
print(Cholesterol)

       1  2  3
id            
0      1  0  0
1      1  0  0
2      1  0  0
3      1  0  0
4      1  0  0
...   .. .. ..
99993  1  0  0
99995  0  1  0
99996  1  0  0
99998  0  1  0
99999  1  0  0

[70000 rows x 3 columns]


## Просмотр метрик после добавления новых признаков

# Мусор

In [34]:
#Общее количество значений класса 1 в наборе данных (TP, FP), общее количество строк в наборе данных
target.sum(), df.shape[0]
#Соотношение количества записей с классом 1 к общему количеству записей:
target.sum()/df.shape[0]
#Eсли отнести все предстказания к классу 0, то accuracy будет равен:
accuracy = (0 + target.shape[0] - target.sum()) / (0 + target.shape[0] - target.sum() + 0 + target.sum())
print("Accuracy = ", accuracy)

Accuracy =  0.5003


начение accuracy, полученное с помощью дерева решений, всего на 0.003657942678200188 больше, чем если бы все предсказания были отнесены к классу 0. Такая ситуация возникает из-за значительного дисбаланса классов (0 намного больше, чем 1). И в подобных ситуациях метрика accuracy не показательна.

